<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/WOS_SCI_SCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WOS+SCI+SCP+PTJ+GS+LNS
Merge the bibliographic datasets for 
* Web of Science, 
* Scielo 
* Scopus 
* Google Scholar
* Puntaje
* Lens
of the scientific articles of Universidad de Antioquia

For details see [merge.ipynb in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/merge.ipynb)

## functions

In [1]:
import unidecode
def get_author_info(x):
    sep='; '
    authors=[{'WOS_author':x[0].split(sep)[0],'affiliation':[x[0].split(sep)[-1]],'i':0}]
    iau=1
    for y  in x:
        y2=y.replace('[','').replace('] ',sep).split(sep)
        for z in y2[:-1]:
            aulist=[ d.get('WOS_author') for d in authors]
            if z not in aulist:
                authors.append({'WOS_author':z,'affiliation':[y2[-1]],'i':iau})
                iau=iau+1
            else:
                if y2[-1] not in [ d.get('affiliation') for d in authors if d.get('WOS_author')==z][0]:
                    index_author=[ d.get('i') for d in authors if d.get('WOS_author')==z][0]
                    authors[index_author]['affiliation'].append(y2[-1])
    return authors

def dictionary_list_add_columns(df,df_dl,df_dl_key,df_dl_i,df_columns):
    '''
    For a
     df: Pandas DataFrame 
    with a:
     df_dl: column of list of dictionaries, with
     df_dl_key: dictionary key: e.g x=[{df_dl_key:1},{df_dl_key:2}]
    for the element df_dl_i of the list:
    Update the dictionary with:
        df_dl_key==x[df_dl_i][df_dl_key]
    with the dictionaries { df_columns[i]: df_columns[i].values }
    '''
    dff=df.copy()
    for key in df_columns:
        tmp=dff[df_dl].combine(dff[key],
                func=lambda x,y: y if pd.isna(y) 
                                   else 
                                     [z.update({key:y}) 
                                     if z.get(df_dl_key)==x[df_dl_i][df_dl_key] 
                                     else z 
                                 for z in x  ] )
    return dff

def split_full_names(y,full_name='full_name'):
    """
    From an input dictionary with {full_name:'APPELLIDO1 APPELLIDO2 NOMBRES'}
    Obtain a dictionary with the several name parts.
    """    
    yy=y.get(full_name).title()
    lfn=len(y[full_name].split())
    aps=0
    d={ 'PRIMER APELLIDO':yy.split()[aps] }
    aps=aps+1
    if lfn>=4:
        names=-2
        if lfn==5: # Extra name or last name
            yyy=yy.split()
            ll=pd.np.array( [len(n) for n in yyy ] )
            if ll[3:][ ll[3:]  <= 3 ].shape[0]:
                # last_names first_first_name de(l) second_first_name
                yy=' '.join( [ y for y in yyy if len(y)>=3] )
            else: 
                # first_last name de(l) second_last_name first_names
                tmpll=yyy.pop() # internal memory
                yy=' '.join( yyy )  
        if len( d['PRIMER APELLIDO'] )<=3:
            d['PRIMER APELLIDO']=d['PRIMER APELLIDO']+' '+yy.split()[aps]
            aps=aps+1
            d.update(  {'SEGUNDO APELLIDO':yy.split()[aps]} )
            names=names+1
            
        d.update({'SEGUNDO APELLIDO':yy.split()[aps]})
        aps=aps+1
        if len( d['SEGUNDO APELLIDO'] )<=3:
            d['SEGUNDO APELLIDO']=d['SEGUNDO APELLIDO']+' '+yy.split()[aps]
            if names==-2:
                names=names+1
    elif lfn>=3:        
        d.update({'SEGUNDO APELLIDO':yy.split()[aps]})
        names=-1
    else: #Colombian interpretation (TODO: Includes Brazilian interpretation)    
        names=-1
    d.update({'NOMBRES':' '.join( yy.split()[names:]),
              'INICIALES':' '.join( [z[0]+'.' for z in yy.split()[names:]] ),
              })
    if not d.get('SEGUNDO APELLIDO'):
        d['SEGUNDO APELLIDO']=''
    #if not d.get('NOMBRE COMPLETO'):
    #    d['NOMBRE COMPLETO']=''        
    return d

# Creates mask Search key in a list of dictionay
# First apply convert null values to string
# Second apply: implement a mask
def find_key_in_list_of_dictionaries(df,column,key,pattern):
    return df[column].apply(lambda x: 
                [ '' if pd.isnull( y.get(key)) else y for y in x ]  ).apply(
                            lambda x: 
                [ True if y.get(key).find(pattern)>-1 else False for y in x  ][0]  )

def key_contains_in_list_of_dictionaries(df,pattern,column='authors_WOS',key='WOS_author'):
    #TODO: loop in column len
    i=0
    r=df[ df[column].str[i].apply(lambda x: {} if pd.isnull(x) else x).apply(
                       lambda x: x.get(key) if x else '').str.contains(
        pattern) ][column].reset_index(drop=True)
    return r

def update_institutional_authors(kkn,AU,authors_column='UDEA_authors',authors_column_key='full_name',
                                        AU_first_last_name='PRIMER APELLIDO',
                                        AU_second_last_name='SEGUNDO APELLIDO',
                                        AU_first_names='NOMBRES'
                                ):
    '''
    For a Data base containing full names of the authors of the articles: SIU,
    include detailed information from other database                    : AU.
      authors_column    : kkn Column with dictionary to be updated
      authors_column_key: Key with full name as a value in  authors_column of kkn
      For Spaniard name, e.g "Juan Pedro Restrepo Correa"
      AU_first_last_name: AU column with "Restrepo"
      AU_second_last_name: AU column with "Correa"
      AU_first_names: AU column with "Juan Pedro"
      full_name: Colum in AU with full name
      
    '''
    full_name='name_tmp'
    AU_columns=list( AU.columns.values )

    AU[full_name]=(AU[AU_first_last_name]+' '+AU[AU_second_last_name]+' '+AU[AU_first_names]
                  ).str.lower().str.strip().apply( unidecode.unidecode )

    maxau=kkn[authors_column].apply(lambda x: [y.get(full_name) for y in x ]).apply(len).max()

    newcolumns=[full_name]+AU_columns
    for i in range(maxau):
        print(i)
        kkn[full_name]=kkn[authors_column].apply(lambda x: [y.get(authors_column_key) for y in x ]
                            ).str[i].apply( lambda x: unidecode.unidecode( x.lower().strip()) 
                                                      if not pd.isna(x) else x)
        if not kkn[~kkn[full_name].isna()].empty:
            kkn=kkn.merge(AU[newcolumns],on=full_name,how='left').reset_index(drop=True)
            kkn=dictionary_list_add_columns(kkn,authors_column,authors_column_key,i,AU_columns)
            kkn=kkn.drop(newcolumns,axis='columns')
    return kkn

In [2]:
import wosplus as wp
import numpy as np
import pandas as pd
import unidecode
import os
import re
import sys
from IPython.display import clear_output
pd.set_option('display.max_colwidth',200)

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [3]:
%%writefile drive.cfg
[FILES]
UDEA_WOS.xlsx       = 1px2IcrjCrkyu7t78Q7PAE5nzV_yuPt9t
UDEA_SCI.xlsx       = 1pWMY5P72j0Ca6D-cm7dn7Q4TBGTs4PWV
UDEA_SCP.xlsx       = 1ulCsFHzDiTmuL9TH8F58ulh0u8Z2ylKh
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj
WOS_SCI_SCP_PTJ.json=1RTDCh5pl0vapjJT_e9ZwadHPGBKGGv6Y
WOS_SCI_SCP_PTJ.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
WOS_SCP_UDEA_SJR_SIU.xlsx=0BxoOXsn2EUNIQ3R4WDhvSzVLQ2s
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs
UDEA_authors_with_WOS_info.json=1o1eVT4JD0FMMICq_oxrTJOzWh47veBMw
produccion_fecha_vig_2003_2018.xlsx=1WbtX4K__TTLxXRjuLvqUYz9tuHCIlS5v
Sample_WOS.xlsx = 1--LJZ4mYyQcaJ93xBdbnYj-ZzdjO2Wq2
UDEA_WOS_SCI_SCP_PTJ.json.gz=1SI2s0IkMnddgrUeR1ssVLFjSvsM1Mnyc
UDEA_WOS_SCI_SCP_PTJ.json=1OkVytKbxJwGvXZDkynkSoUDtkUOTaT4A

Overwriting drive.cfg


##  Load data bases

In [4]:
affil='Univ Antioquia'
drive_files=wp.wosplus('drive.cfg')

#### DEBUG: if False stop in UDEA_PTJ!!!!

In [5]:
UDEAjsonfile='WOS_SCI_SCP_PTJ.json.gz'
if os.path.exists(UDEAjsonfile):
    UDEA=               pd.read_json(UDEAjsonfile,compression='gzip').reset_index(drop=True)
else:    
    UDEA=drive_files.read_drive_json(UDEAjsonfile,compression='gzip').reset_index(drop=True)

In [6]:
for t in UDEA.Tipo.unique():
    print( '{}:{}'.format( t, UDEA[ UDEA.Tipo==t].shape[0] ) )

WOS:1646
WOS_SCP:5268
SCP:2157
WOS_UDEA:238
WOS_SCP_UDEA:553
WOS_SCI_SCP_UDEA:129
SCI:2396
SCI_SCP_UDEA:372
SCP_UDEA:418
WOS_SCI_UDEA:37
SCI_UDEA:497
WOS_SCI_SCP:639
SCI_SCP:1244
WOS_SCI:110


In [7]:
UDEA.shape

(15704, 181)

In [8]:
AU=drive_files.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')

## Extract  affiliation from C1

In [9]:
#Prepare UDEA.C1
UDEA['C1']=UDEA['C1'].apply(lambda s: s if s!='' else None)
UDEA_SCI=UDEA.SCI_C1.combine(UDEA['Tipo'],func=lambda x,y: x if y.find('SCI')>-1 and y.find('WOS')==-1 else None)
UDEA_SCI=UDEA_SCI.apply(lambda x:x.replace(
    'Universidad de Antioquia',affil).replace(
    'Univ. de Antioquia',affil) if not pd.isnull(x) else x)
#Update only if not filled already
UDEA['C1']=UDEA_SCI.combine(UDEA['C1'],lambda x,y:x if pd.isnull(y) else y)

In [10]:
#Update C1 from SCP if neccesary
rename='[\w\-\.\s]+'
SCP2WOS=UDEA['SCP_Authors with affiliations'].str.replace(
                                    '; ','\n').str.replace(
                                    '^({},{}),'.format( rename,rename),r'[\1]',re.UNICODE).str.replace(
                                    '\n({},{}),'.format(rename,rename),r'\n[\1]',re.UNICODE).str.replace(
                                    '(,\s+\w\.)(\w\.\])'.format(rename,rename),r'\1 \2',re.UNICODE)
i=0
SCP2WOS.loc[i] 

''

In [11]:
# Normalize to WOS 
SCP2WOS=SCP2WOS.str.replace('Universi[dadty]{2,3}\s+[deofDEOF]{2}\s+Antioqu[ií]a',affil
      ).str.replace('U[\.niv]{1,4}\s+[deofDEOF]{0,2}\s*Antioqu[ií]a',affil
      ).str.replace('Antioqu[ií]a\s+[deofDEOF]{0,2}\s*Universi[dadty]{2,3}',affil
      )
UDEA_SCP=SCP2WOS.combine(UDEA['Tipo'],func=lambda x,y: x if y=='SCP' else None)
#Update only if not filled already
UDEA['C1']=UDEA_SCP.combine(UDEA['C1'],lambda x,y:x if pd.isnull(y) else y)
UDEA[UDEA['C1'].isnull()].shape

(30, 181)

In [12]:
UDEA[UDEA.Tipo=='WOS'].reset_index(drop=True).C1.loc[0]

'[Duque, Luis F.; Montoya, Nilton; Restrepo, Alexandra] Univ Antioquia, Sch Publ Hlth, Medellin, Colombia.\n'

In [13]:
UDEA['authors_WOS']=UDEA.C1.apply(lambda x: x.split('\n') if x else x).apply(
    lambda x:   [y.replace('[','').replace('] ','; ') for y in x if y.find(affil)>-1 ] if x else x ).apply(
     lambda x: get_author_info(x) if x else x)

# Improve normalization: remove C1s with only affiliation (from Scielo)
UDEA['authors_WOS']=UDEA['authors_WOS'].apply( 
    lambda x: [d for d in x if d.get('WOS_author').find(affil)==-1] if type(x)==list else x )

In [14]:
UDEA[UDEA.Tipo=='SCP'].reset_index(drop=True).loc[0].authors_WOS

[{'WOS_author': 'Márquez Arabia, J. J.',
  'affiliation': ['Instituto Universitario de Educación Física, Univ Antioquia, Medellín, Colombia'],
  'i': 0},
 {'WOS_author': 'Ramón Suárez, G.',
  'affiliation': ['Instituto Universitario de Educación Física, Univ Antioquia, Medellín, Colombia'],
  'i': 1}]

## Load trained old data 

In [15]:
MERGE_WITH_TRAINED=False
if MERGE_WITH_TRAINED:
    if os.path.exists('WOS_SCP_UDEA_SJR_SIU.xlsx'):
        SIU=pd.read_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')
    else:    
        SIU=drive_files.read_drive_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')

    SIU.Tipo.unique()

    SIU=SIU[SIU.Tipo.str.contains('\+UDEA')].reset_index(drop=True)

    SIU['UDEA_autores']=SIU['UDEA_autores'].str.replace('\s+',' ')

    SIU['UDEA_authors']=SIU.UDEA_autores.str.split(';').apply(lambda x: [{'full_name':y} for y in x ])

## Merge with official researcher list

In [16]:
if MERGE_WITH_TRAINED:
    kkn=SIU.copy()
    
    AU=drive_files.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')
    kkn=update_institutional_authors(kkn,AU)

    print(kkn.shape,SIU.shape)

    SIU=kkn.copy()

Quality check

In [17]:
if MERGE_WITH_TRAINED:
    SIU[ find_key_in_list_of_dictionaries(SIU,'UDEA_authors','full_name','ZAPATA') ].UDEA_authors.loc[241]

## Merge WOS_SCP_SCI with trained data set PTJ_CTR

Merge requires split in DI and TI

In [18]:
if MERGE_WITH_TRAINED:
    #Be sure that always be True. The columns will be obtaied from SIU
    REMOVE_UDEA_columns=True
    if REMOVE_UDEA_columns:
        UDEA_columns=[c for c in UDEA.columns if c.find('UDEA_')>-1]
        UDEA=UDEA.drop(UDEA_columns,axis='columns')

    udea_columns=[       'UDEA_autores',
           'UDEA_año realiz', 'UDEA_doi', 'UDEA_fecha aplicación',
           'UDEA_idioma', 'UDEA_item adic', 'UDEA_material', 'UDEA_nombre',
           'UDEA_nombre revista o premio', 'UDEA_nro autores', 'UDEA_país',
           'UDEA_procodigo', 'UDEA_ptos', 'UDEA_simple_doi', 'UDEA_título',
           'UDEA_valor item','UDEA_authors']

    SIUDI=SIU[~SIU.DI.isna()].drop_duplicates('DI').reset_index(drop=True)
    SIUTI=SIU[ SIU.DI.isna()].drop_duplicates('TI').reset_index(drop=True)
    SIUTI=SIUTI[SIUTI!=''].reset_index(drop=True)
    SIUTI=SIUTI[~SIUTI.TI.isnull()].reset_index(drop=True)
    SIUTI=SIUTI[ SIUTI.TI.apply(len)>20 ].reset_index(drop=True)

    UDEADI=UDEA[UDEA.DI!=''].drop_duplicates('DI').reset_index(drop=True)
    UDEATI=UDEA[UDEA.DI==''].drop_duplicates('TI').reset_index(drop=True)

    UDEA_mergeDI=UDEADI.merge( SIUDI[ ['DI']+udea_columns ],on='DI',how='left' )

    UDEA_PTJ=pd.DataFrame()
    UDEA_PTJ_NOT=pd.DataFrame()
    UDEA_PTJ=UDEA_mergeDI[~UDEA_mergeDI.UDEA_autores.isna()].reset_index(drop=True)
    UDEA_PTJ_NOT=UDEA_mergeDI[UDEA_mergeDI.UDEA_autores.isna()].reset_index(drop=True)

    UDEATI['tmptitle']=UDEATI.TI.str.strip()
    SIUTI['tmptitle']=SIUTI.TI.str.strip()

    kk=UDEATI.merge( SIUTI[ ['tmptitle']+udea_columns ],on='tmptitle',how='left' ).drop('tmptitle',axis='columns')

    UDEA_PTJ=UDEA_PTJ.append( kk[ ~kk.UDEA_autores.isna() ] ).reset_index(drop=True)
    UDEA_PTJ_NOT=UDEA_PTJ_NOT.append( kk[ kk.UDEA_autores.isna() ] ).reset_index(drop=True)

    print(UDEA_PTJ.shape[0]+UDEA_PTJ_NOT.shape[0],UDEA.shape)

    print(UDEA_PTJ.shape,UDEA_PTJ_NOT.shape)

    UDEA=UDEA_PTJ.append(
        UDEA_PTJ_NOT).reset_index(
        drop=True)

In [19]:
key_contains_in_list_of_dictionaries(UDEA,'Restrepo, D',column='authors_WOS',key='WOS_author').loc[1:2]

1    [{'i': 0, 'WOS_author': 'Granda-Restrepo, Diana', 'affiliation': ['Univ Antioquia, Fac Quim Farmaceut, Dept Alimentos, Grp Invest Biotecnol Alimentos BIOALI, Medellin, Colombia.']}, {'i': 1, 'WOS_...
2    [{'i': 0, 'WOS_author': 'Restrepo, Diego', 'affiliation': ['Univ Antioquia, Inst Fis, Calle 70 52-21,Apartado Aereo 1226, Medellin, Colombia.']}, {'i': 1, 'WOS_author': 'Zapata, Oscar', 'affiliati...
Name: authors_WOS, dtype: object

In [20]:
if MERGE_WITH_TRAINED:
    UDEA.to_json('UDEAtmp.json')
    RECOVER=False
    if RECOVER:
        UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

## Add `UDEA.authors_WOS` info* within `UDEA.UDEA_authors` data**
(\*) obtained from `UDEA.C1`

(\*\*) Obtained from [puntaje trained old UDEA data](./WOS_SCI_SCP_PTJ_GS_LNS.ipynb#Merge-with-trained-data-set) and the [official researcher list](./WOS_SCI_SCP_PTJ_GS_LNS.ipynb#Merge-with-official-researcher-list)

Obtain name parts and initials from full name in `UDEA_authors` dictionary and update `UDEA_authors` with them

In [21]:
import sys
if 'UDEA_authors' not in UDEA.columns:
    sys.exit('Make MERGE_WITH_TRAINED True and run again')

In [22]:
print 1

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-22-c94594b6b28f>, line 1)

In [ ]:
# tmp is for update dict
dictupdatetmp=UDEA['UDEA_authors'].apply(lambda x: [y.update( 
                split_full_names(y,full_name='full_name')  ) if not pd.isnull(
                y.get('full_name')) else y for y in x] 
                                   if type(x)==list 
                                   else x)

In [ ]:
x,y=UDEA['authors_WOS'].combine( UDEA['UDEA_authors'], func=lambda x,y:(x,y) ).loc[92]

In [ ]:
x

In [ ]:
y

In [ ]:
def wos_names_append(wos_names,last_name,first_names,initials):
    wos_names=wos_names+[           last_name+', '+first_names]
    if len( first_names.split())>1:
        wos_names=wos_names+[ last_name+', '+first_names.split()[0] ]
    if len( first_names.split())==2 and  len(first_names.split()[-1]):
        wos_names=wos_names+[ last_name+', '+first_names.split()[-1] ]
    wos_names=wos_names+[ last_name+', '+initials]
    if len( initials.split())>1:
        wos_names=wos_names+[ last_name+', '+initials.split()[0]]
    if len(initials.split())==2:
        wos_names=wos_names+[
              last_name+', '+first_names.split()[0]+' '+initials.split()[-1] ]
        wos_names=wos_names+[last_name+', '+initials.split()[-1] ]
    return wos_names
    
def wos_names_list(dy ,y_keys=['PRIMER APELLIDO','NOMBRES','INICIALES','SEGUNDO APELLIDO','full_name']   ):
    last_name= unidecode.unidecode( dy[y_keys[0]]   )
    first_names=unidecode.unidecode( dy[y_keys[1]]  )
    initials=unidecode.unidecode( dy[y_keys[2]]  )

    wos_names=[]
    wos_names=wos_names_append(wos_names,last_name,first_names,initials)
    
    if dy.get( y_keys[3] ):
        last_names= unidecode.unidecode( dy[y_keys[0]]+'-'+dy[y_keys[3]]   )
        wos_names=wos_names_append(wos_names,last_names,first_names,initials)
    return wos_names
    
def combinewos(x,y,x_keys=['WOS_author','affiliation'],
                   y_keys=['PRIMER APELLIDO','NOMBRES','INICIALES','SEGUNDO APELLIDO','full_name'],
                   xy_keys=['WOS_author','WOS_affiliation']):
    if type(x)==list and type(y)==list:
        for dx in x:
            wos_name=unidecode.unidecode( dx[ x_keys[0] ] )
            WOS_affiliation= dx[x_keys[1]]
            # Try by buildinng spanish-like names list                                
            for i in range( len(y) ):
                if wos_name.title() in wos_names_list(y[i] ,y_keys):
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break
            #Try again but comparing full lists
            wos_name_to_list=wos_name.replace(',','').replace('-',' ').title().split()
            for i in range( len(y) ):
                yi_to_list=unidecode.unidecode( y[i][y_keys[4]].title() )
                if yi_to_list:
                    yi_to_list=yi_to_list.split()
                else:
                    yi_to_list=[]
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break
            #Try again but comparing full lists with initials
            wos_name_to_list=wos_name.replace(',','').replace('-',' ').title().split()
            for i in range( len(y) ):
                yi_to_list=[y[i][y_keys[0]],y[i][y_keys[3]] ]+y[i][y_keys[2]].split()
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break
            #Try again but comparing full lists with first first name and initial
            for i in range( len(y) ):
                yi_to_list=[y[i][y_keys[0]],y[i][y_keys[3]],y[i][y_keys[1]].split()[0],
                              y[i][y_keys[2]].split()[-1]]
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break                    
            #Try again but comparing full lists with second first name and initial
            for i in range( len(y) ):
                yi_to_list=[y[i][y_keys[0]],y[i][y_keys[3]],y[i][y_keys[1]].split()[-1],
                              y[i][y_keys[2]].split()[0]]
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break                    
                    
                    
    return y

In [ ]:
wos_names_list( split_full_names( y[0] ) )

In [ ]:
#kk=UDEA['UDEA_authors'].str[0].apply( lambda d: d if type(d)==dict else pd.np.nan ).dropna()
#kk[kk.apply( lambda d: d.get('full_name') if type(d)==dict else pd.np.nan ).isna()]

In [ ]:
kk=UDEA['authors_WOS'].combine( UDEA['UDEA_authors'], func=combinewos )

Combines the two list into a single one in  UDEA_authors

check

Full query of auhor_WOS in UDEA_authors:

In [ ]:
UDEA['UDEA_authors'].apply(lambda x: [y  for y in x ][0] if type(x)==list else x).loc[92]

In [ ]:
UDEA.to_json('UDEAtmp.json')

### Load output restuls of previous Cell runs

In [ ]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [ ]:
def extract_internal_value_of_a_dictionary_key_in_a_list_of_dictionaries(df,
    list_of_dictionaries='UDEA_authors',
    dictionary_key='WOS_author'):
    #Extract internal value of a dictionary key in a list of dictionaries and empty list otherwise
    
    return df[list_of_dictionaries].apply(lambda x: [y.get(dictionary_key) 
                                                        if y.get(dictionary_key) else []   
                                                        for y in x  ]
                           if type(x)==list else [])

def extract_internal_list_as_value_of_a_dictionary_key_in_a_list_of_dictionaries(df,
    list_of_dictionaries='UDEA_authors',
    dictionary_key='WOS_author'):
    #Extract internal list as value of a dictionary key in a list of dictionaries and empty list otherwise
    
    return extract_internal_value_of_a_dictionary_key_in_a_list_of_dictionaries(df,
                    list_of_dictionaries,dictionary_key).apply(lambda x: 
                           [item for sublist  in x for item in sublist] 
                            if type(x)==list else x)

def mask_on_internal_value_of_a_dictionary_key_in_a_list_of_dictionaries(df,
    pattern='RESTREPO QUINTERO DIEGO ALEJANDRO',
    list_of_dictionaries='UDEA_authors',dictionary_key='full_name'):
    """
    Build a mask for a Pandas Series of list of dictionaries of label:
      list_of_dictionaries. 
    The:
      dictionary_key must be a single value like string or float
    """
    return extract_internal_value_of_a_dictionary_key_in_a_list_of_dictionaries(df,
            list_of_dictionaries,dictionary_key).apply( 
            lambda x: pattern in x)

In [ ]:
pattern='Restrepo, Diego'
mask=extract_internal_list_as_value_of_a_dictionary_key_in_a_list_of_dictionaries(
            UDEA,list_of_dictionaries='UDEA_authors',dictionary_key='WOS_author').apply( 
            lambda x: pattern in x)
UDEA[mask]['UDEA_authors'].reset_index(drop=True)

In [ ]:
pattern='Restrepo, D.'
mask=extract_internal_list_as_value_of_a_dictionary_key_in_a_list_of_dictionaries(
            UDEA,list_of_dictionaries='UDEA_authors',dictionary_key='WOS_author').apply( 
            lambda x: pattern in x)
UDEA[mask]['UDEA_authors'].reset_index(drop=True)

In [ ]:
pattern

In [ ]:
mask=extract_internal_list_as_value_of_a_dictionary_key_in_a_list_of_dictionaries(
            UDEA,list_of_dictionaries='UDEA_authors',dictionary_key='WOS_author').apply( 
            lambda x: pattern in x)
UDEA[mask]['UDEA_authors'].reset_index(drop=True)

In [ ]:
def extract_dictionary_from_list(df,list_name,i=0):
    return df[list_name].apply( lambda l: [ d for d in l][i] if type(l)==list else l )
def extract_key_value_from_series(ds,key):
    return ds.apply( lambda d: d.get(key)if type(d)==dict else d  )

In [ ]:
mask=extract_key_value_from_series( 
    extract_dictionary_from_list( UDEA,'UDEA_authors',i=0),'WOS_author').str[0].str.contains('\-').fillna(False)
UDEA[mask]['UDEA_authors'].reset_index(drop=True).str[0].loc[1]

TODO: Extract dict or value

## Build a single profile for all

flatten a list of lists:

In [ ]:
ua=pd.DataFrame()

In [ ]:
aumax=UDEA['UDEA_authors'].dropna().apply(len).max() 
ua=pd.DataFrame()
for i in range(aumax):
    kkk=pd.DataFrame()
    kkk['UDEA_authors']= UDEA['UDEA_authors'].str[i].dropna()
    kkk['authors_WOS']= UDEA['authors_WOS']
    kkk['SCP_Authors']=UDEA['SCP_Authors']
    kkk['tmp_str']=kkk['UDEA_authors'].astype(str)
    kkk=kkk.drop_duplicates('tmp_str')
    ua=ua.append(kkk).reset_index(drop=True)
    #ua['UDEA_authors']= 

In [ ]:
ua['tmp_author']=ua['UDEA_authors'].apply( lambda d: d.get('full_name') )

BUG: `UDEA_authors` without `WOS_author key`

### Filter identified WOS articles

In [ ]:
ua['authors_WOS']=ua['authors_WOS'].apply(lambda l: l if l else pd.np.nan)
ua=ua[~ua['authors_WOS'].isna()].reset_index(drop=True)

In [ ]:
full_names=ua['tmp_author'].unique()

In [ ]:
full_names[:3]

In [ ]:
ua.shape,full_names.shape

TODO: implement also SCP authors

Example

In [ ]:
aunly=pd.DataFrame()
for f in full_names:
    clear_output(wait=True)
    print(f)    
    kk=pd.DataFrame( { 'tmp_author':[f]  } ).merge(
          ua[['tmp_author','UDEA_authors']],on='tmp_author',how='left')

    kk['tmp_str']=kk.UDEA_authors.astype(str)

    kk=kk.drop_duplicates('tmp_str').dropna()#[['tmp_author','UDEA_authors']]

    try:
        laff=list( kk['UDEA_authors'].apply(lambda d: d.get( 'WOS_affiliation' )
                                     ).dropna().apply(pd.Series).stack().unique() )
        lau=list( kk['UDEA_authors'].apply(lambda d: d.get( 'WOS_author' )
                                     ).dropna().apply(pd.Series).stack().unique() )
    except AttributeError:
        laff=[];lau=[]

    if len(laff)>0 and len(lau)>0:
        tmpupdate=kk['UDEA_authors'].apply(lambda d: d.update({'WOS_author':lau,'WOS_affiliation':laff}) )

        kk['tmp_str']=kk['UDEA_authors'].astype(str)

        kk=kk.drop_duplicates('tmp_str')

        kk['tmp_len']=kk['tmp_str'].apply(len)#.astype(str)

        aunly=aunly.append( kk.sort_values('tmp_len',ascending=False).drop(index=kk.index[1:]).drop(
               ['tmp_str','tmp_len'],axis='columns') ).reset_index(drop=True)

In [ ]:
#DEBUG
aunly[aunly.tmp_author.fillna('').str.contains('DUQUE ECHEVERRI CARLOS ALBERTO')
      ].reset_index(drop=True).UDEA_authors.loc[0]

In [ ]:
ua['UDEA_authors'].loc[0]

In [ ]:
ua['authors_WOS'].loc[0]

In [ ]:
aunly.shape

In [ ]:
aunly.to_json('UDEA_authors_with_WOS_info.json')

In [ ]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [ ]:
UDEA.shape

In [ ]:
if RECOVER:
    if os.path.exists('UDEA_authors_with_WOS_info.json' ):
        aunly=pd.read_json('UDEA_authors_with_WOS_info.json')
    else:
        aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json')

In [ ]:
aunly.shape

(800, 2)

## Merge UDEA with authors

In [ ]:
# loop upon i and j to merge aunly in UDEA such that the full authors and affiliations are registered
#UDEA.merge

In [ ]:
def fill_full_wos_author_info(l,WOS_df=aunly,full_name='full_name',full_name_column='tmp_author',
                               WOS_column='UDEA_authors',WOS_author='WOS_author',
                               WOS_affiliation='WOS_affiliation'):
    newl=[]
    if type(l)==list:
        for d in l:
            if d.get('WOS_author'):
                #find in aunly
                mtch=WOS_df[WOS_df[full_name_column]==d.get(full_name)].reset_index(drop=True)
                if mtch.shape[0]==1:
                    #update d
                    if mtch[WOS_column].loc[0].get(WOS_author):
                        d[WOS_author]=mtch[WOS_column].loc[0].get(WOS_author)
                        d[WOS_affiliation]=mtch[WOS_column].loc[0].get(WOS_affiliation)
            newl.append(d)
    else:
        newl=l
    return newl        

In [ ]:
#l=UDEA['UDEA_authors'].loc[4]
#fill_full_wos_author_info(l,aunly)[0].get('WOS_author')

In [ ]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(fill_full_wos_author_info )

In [ ]:
UDEA.to_json('UDEAtmp.json')

In [ ]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [ ]:
UDEA.shape

## Implementation without merge

In [ ]:
import Levenshtein as lv
def find_author_affiliation(author,affiliation,author_df=aunly,column='UDEA_authors',
                            author_key='WOS_author',affiliation_key='WOS_affiliation',ratio=0.9):
    '''
    find the WOS+"UDEA puntaje" dictionary for WOS author:   `author`
    and WOS affiliation:                                    `affiliation`
    The information is  searched in 
    WOS+"UDEA puntaje" DataFrame:                            `author_df`, 
    which has the column:                                    `column` 
    which contains a dictionary with list value for the key: `author_key`
    and list value for the key:                              `affiliation_key`.
    Affiliation must be similar until a Levenshtein ratio:   `ratio`
    '''    
    au=author_df[ author_df[column].apply(
                lambda d: d.get(author_key) if type(d)==dict else '').apply(
                  lambda l: author in l)]#.reset_index(drop=True).loc[0,column]
    if au.shape[0]>0:
        #Fast
        auf=au[au[column].apply(
                 lambda d: d.get(affiliation_key) if type(d)==dict else '').apply(
                 lambda l: affiliation in l)]
        

        if auf.shape[0]>0:
              return auf.reset_index(drop=True).loc[0,column]          
        #Slow
        else:
            aus=au[au[column].apply(
                 lambda d: d.get(affiliation_key) if type(d)==dict else '').apply(
                 lambda l: len( [af for af in l if lv.ratio(af,affiliation) > ratio ] )>0 )]

            if aus.shape[0]==1: #fix 1 to avoid homonymous
                dold=aus.reset_index(drop=True).loc[0,column]
                # Dictionary is automatically updated in author_df!
                dold[affiliation_key]=dold[affiliation_key]+[affiliation]
                return dold
    else:
        return None

def get_UDEA_authors(x,y,x_author_key='WOS_author',x_affiliation_key='affiliation',
                        author_df=aunly,column='UDEA_authors',
                        author_key='WOS_author',affiliation_key='WOS_affiliation',
                        ratio=0.9):
    '''
    get the WOS+"UDEA puntaje" list of dictionaries for WOS author list 
    and affiliation list in the  list of dictionaries:      `x`, 
    where each dictionary have the string value for the key: `x_author_key`, 
    and the list value for the key:                          `x_affiliation_key`.
    The information is obtained directly from the 
    WOS+"UDEA puntaje" list:                                 `y` 
    if already there, or searched in 
    WOS+"UDEA puntaje" DataFrame:                            `author_df`, 
    which has the column:                                    `column` 
    which contains a dictionary with list value for the key: `author_key`
    and list value for the key:                              `affiliation_key`.
    If not foun None is returned.
    WOS info can be changed for other standarized dababase info
    and "UDEA puntaje" can be changed from any other full name author
    and affiliation info.
    
    IMPORTANT:
    The list of values in:                                   `affiliation_key` 
    is automatically updated with the similar first 
    affiliation value of the list in:                        `x_affiliation_key`
    according with the Levenshtein similarity ratio:         `ratio`
    '''
    if type(y)==list:
        #already filled:
        return y

    au=[]
    if ( type(x)==list and x):
        for j in range(len(x)):
            xx=find_author_affiliation(x[j].get(x_author_key),x[j].get(x_affiliation_key)[0],
                                        author_df=author_df,
                                        author_key=author_key,
                                        affiliation_key=affiliation_key,
                                        ratio=ratio )
            if xx:
                au.append(xx)
    if au:
        return au
    else:
        return None

In [ ]:
i=6
x=UDEA['authors_WOS'].loc[i]
y=UDEA['UDEA_authors'].loc[i]

In [ ]:
x

In [ ]:
y

In [ ]:
get_UDEA_authors(x,y)

In [ ]:
kk=UDEA.authors_WOS.combine(UDEA.UDEA_authors,func=get_UDEA_authors)

In [ ]:
UDEA.UDEA_authors.dropna().shape

In [ ]:
UDEA['UDEA_authors']=kk

In [ ]:
UDEA.UDEA_authors.dropna().shape

In [ ]:
aunly.to_json('UDEA_authors_with_WOS_info.json')

In [ ]:
RECOVER=False
if RECOVER:
    if os.path.exists('UDEA_authors_with_WOS_info.json' ):
        aunly=pd.read_json('UDEA_authors_with_WOS_info.json')
    else:
        aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json')

In [ ]:
UDEA.to_json('UDEAtmp.json')

In [ ]:
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

### Test

In [ ]:
qq=UDEA.copy()
qq['UDEA_authors']=None

In [ ]:
qqq=qq.authors_WOS.combine(qq.UDEA_authors,func=get_UDEA_authors)

In [ ]:
qq=UDEA.copy()
qq['NEW']=qqq
qq['EXTRA']=kk

In [ ]:
(qq[qq.UDEA_authors.isna()][['NEW','EXTRA']].dropna(subset=['NEW']))[qq.EXTRA==None].shape

# Puntaje

UDEA

In [ ]:
#TODO: pass pandas.load args
tmp=drive_files.load_biblio('produccion_fecha_vig_2003_2018.xlsx',prefix='UDEA')

Normalize title translation

In [ ]:
import re

In [ ]:
septrans=r'(.+)\((.{10,})\)$'

In [ ]:
drive_files.biblio['UDEA']['UDEA_título_list']=drive_files.biblio['UDEA'].UDEA_título.str.replace(
    septrans,r'\1;;\2',re.UNICODE).str.split(';;').apply(
   lambda l: [ re.sub( r'^"','',
                      re.sub( r'"$','', s.strip() )
                     ) for s in l] )#.loc[i]

In [ ]:
drive_files.biblio['UDEA']['UDEA_DOI']=''

In [ ]:
drive_files.biblio['UDEA']=drive_files.biblio['UDEA'].reset_index(drop=True)

In [ ]:
drive_files.biblio['UDEA'].Tipo.unique()

Remove duplicated rows

In [ ]:
i=0 #1
titlec='UDEA_TI'
authorc='UDEA_nombre'
authorsc='UDEA_nombres'
drive_files.biblio['UDEA']['UDEA_TI']=drive_files.biblio['UDEA'].UDEA_título_list.str[i]

In [ ]:
drive_files.biblio['UDEA'].shape

In [ ]:
multi_au=drive_files.biblio['UDEA'][drive_files.biblio['UDEA'].duplicated(subset=[titlec],keep=False)]

In [ ]:
single_au=drive_files.biblio['UDEA'][~drive_files.biblio['UDEA'].duplicated(subset=[titlec],keep=False)]

In [ ]:
multi_au.shape[0]+single_au.shape[0]

In [ ]:
multi_au=multi_au.sort_values(titlec).reset_index(drop=True)
multi_au[[authorc,titlec]][:7]

In [ ]:
t_old=''
au_old=pd.np.nan
for i in multi_au.index:
    t=multi_au.loc[i,titlec]
    if t==t_old:
        au_old=au_old+';'+multi_au.loc[i,authorc]
        multi_au.loc[i-1,authorsc]=pd.np.nan
    else:
        t_old=t
        multi_au.loc[i-1,authorsc]=au_old
        au_old=multi_au.loc[i,authorc]

In [ ]:
multi_au[[authorc,authorsc,titlec]][:7]

In [ ]:
multi_au.shape[0],multi_au.dropna(subset=[authorsc]).shape[0]

In [ ]:
drive_files.biblio['UDEA']=(multi_au.dropna(subset=[authorsc]).append(single_au,sort=False)
                           ).reset_index(drop=True)
drive_files.biblio['UDEA'].shape

WOS

In [ ]:
RECOVER=True
if RECOVER:
    tmp=drive_files.load_biblio('UDEA_WOS_SCI_SCP_PTJ.json')# TODO CHANGE FOR LAST VERSION IN GOOGLE DRIVE
#DEBUG
#tmp=drive_files.load_biblio('Sample_WOS.xlsx')

In [ ]:
drive_files.WOS.shape

In [ ]:
drive_files.biblio['WOS'].Tipo.unique()

Split file

In [ ]:
drive_files.biblio['WOS']['UDEA_authors']=drive_files.biblio['WOS'].UDEA_authors.apply(lambda x: 
                                x if type(x)==list else None).reset_index(drop=True)

In [ ]:
drive_files.biblio['WOS'].shape

In [ ]:
#ALREADY in PTJ
UDEA_PTJ=drive_files.biblio['WOS'][ ~drive_files.biblio['WOS']['UDEA_authors'].isna()].reset_index(drop=True)
# Missing PTJ. To be proccesses below
UDEA_PTJ_NOT    =drive_files.biblio['WOS'][ drive_files.biblio['WOS']['UDEA_authors'].isna()].reset_index(drop=True)

In [ ]:
UDEA_PTJ.shape, UDEA_PTJ_NOT.shape

In [ ]:
drive_files.biblio['WOS']=UDEA_PTJ_NOT

In [ ]:
udea_columns=[c for c in drive_files.biblio['WOS'].columns if c.find('UDEA_')>-1]

In [ ]:
drive_files.biblio['WOS']=drive_files.biblio['WOS'].drop( udea_columns, axis='columns' )

In [ ]:
drive_files.biblio['WOS'].shape

In [ ]:
DEBUG=False
if DEBUG:
    drive_files.biblio['UDEA']=drive_files.biblio['UDEA'][:10]
    drive_files.biblio['WOS']=drive_files.biblio['WOS'][:10]

drive_files.UDEA=drive_files.biblio['UDEA']
drive_files.WOS =drive_files.biblio['WOS']    

In [ ]:
kk=drive_files.merge(left="WOS", right="UDEA",
                     left_DOI="DI", left_TI="TI",
                     right_DOI="UDEA_DOI", right_TI="UDEA_TI",
                     left_author="AU", left_year="PY",
                     right_author="UDEA_nombre",right_year="UDEA_año realiz",
                     left_extra_journal="SO",
                     right_extra_journal="UDEA_nombre revista o premio"
                     )

Prepare new WOS

In [ ]:
drive_files.biblio['WOS_UDEA'].shape

clean pure PTJ entries

In [ ]:
newwos=drive_files.biblio['WOS_UDEA'][drive_files.WOS_UDEA.Tipo!='UDEA']

In [ ]:
newwos.shape #expected 7094. Ignoring differences

In [ ]:
app_to_UDEA_PTJ=newwos[newwos.Tipo.str.contains('UDEA')].reset_index(drop=True)

In [ ]:
app_to_UDEA_PTJ.shape

In [ ]:
new_UDEA_not_PTJ=newwos[~newwos.Tipo.str.contains('UDEA')].reset_index(drop=True)
new_UDEA_not_PTJ.shape[0]+app_to_UDEA_PTJ.shape[0]

In [ ]:
drive_files.biblio['WOS']=new_UDEA_not_PTJ

drive_files.biblio['WOS']=drive_files.biblio['WOS'].drop( 
     [ c for c in drive_files.biblio['WOS'].columns if c.find('UDEA_')>-1  ],
       axis='columns')
new_UDEA_not_PTJ.shape

Prepare new UDEA

kkk=app_to_UDEA_PTJ[['UDEA_TI','Tipo']].merge(drive_files.biblio['UDEA'],on='UDEA_TI',how='right')
drive_files.biblio['UDEA']=kkk[kkk.Tipo_x.isna()].drop('Tipo_x',axis='columns'
                                              ).rename({'Tipo_y':'Tipo'},axis='columns'
                                              ).reset_index(drop=True)

In [ ]:
i=1
drive_files.biblio['UDEA']['UDEA_TI']=drive_files.biblio['UDEA'].UDEA_título_list.str[i]

drive_files.biblio['UDEA']=drive_files.biblio['UDEA'].dropna(subset=['UDEA_TI']).reset_index(drop=True)

kkk[~kkk.Tipo_x.isna()].shape,drive_files.biblio['UDEA'].shape

In [ ]:
drive_files.biblio['UDEA']=wp.fill_NaN(drive_files.biblio['UDEA'])
drive_files.biblio['WOS'] =wp.fill_NaN(drive_files.biblio['WOS'])    
drive_files.UDEA=drive_files.biblio['UDEA']
drive_files.WOS =drive_files.biblio['WOS']    

In [ ]:
kk=drive_files.merge(left="WOS", right="UDEA",
                     left_DOI="DI", left_TI="TI",
                     right_DOI="UDEA_DOI", right_TI="UDEA_TI",
                     left_author="AU", left_year="PY",
                     right_author="UDEA_nombre",right_year="UDEA_año realiz",
                     left_extra_journal="SO",
                     right_extra_journal="UDEA_nombre revista o premio"
                     )

WOS

In [ ]:
newwos=drive_files.WOS_UDEA[drive_files.WOS_UDEA.Tipo!='UDEA']

In [ ]:
newwos.shape

In [ ]:
app_to_UDEA_PTJ_2=newwos[newwos.Tipo.str.contains('UDEA')].reset_index(drop=True)
app_to_UDEA_PTJ_2.shape

In [ ]:
new_UDEA_not_PTJ=newwos[~newwos.Tipo.str.contains('UDEA')].reset_index(drop=True)
new_UDEA_not_PTJ.shape[0]+app_to_UDEA_PTJ.shape[0]

In [ ]:
miss_2=newwos[~newwos.Tipo.str.contains('UDEA')].reset_index(drop=True)
drive_files.biblio['WOS']=new_UDEA_not_PTJ

drive_files.biblio['WOS']=drive_files.biblio['WOS'].drop( 
     [ c for c in drive_files.biblio['WOS'].columns if c.find('UDEA_')>-1  ],
       axis='columns')
drive_files.biblio['WOS'].shape

UDEA

kkk=app_to_UDEA_PTJ_2[['UDEA_TI','Tipo']].merge(drive_files.biblio['UDEA'],on='UDEA_TI',how='right')
drive_files.biblio['UDEA']=kkk[kkk.Tipo_x.isna()].drop('Tipo_x',axis='columns'
                                              ).rename({'Tipo_y':'Tipo'},axis='columns'
                                              ).reset_index(drop=True)

drive_files.biblio['UDEA']['UDEA_TI']=drive_files.biblio['UDEA'].UDEA_título
drive_files.biblio['UDEA']=drive_files.biblio['UDEA'].dropna(subset=['UDEA_TI']).reset_index(drop=True)
kkk[~kkk.Tipo_x.isna()].shape,drive_files.biblio['UDEA'].shape

In [ ]:
drive_files.biblio['UDEA']=wp.fill_NaN(drive_files.biblio['UDEA'])
drive_files.biblio['WOS'] =wp.fill_NaN(drive_files.biblio['WOS'])    
drive_files.UDEA=drive_files.biblio['UDEA']
drive_files.WOS =drive_files.biblio['WOS']

In [ ]:
kk=drive_files.merge(left="WOS", right="UDEA",
                     left_DOI="DI", left_TI="TI",
                     right_DOI="UDEA_DOI", right_TI="UDEA_título",
                     left_author="AU", left_year="PY",
                     right_author="UDEA_nombre",right_year="UDEA_año realiz",
                     left_extra_journal="SO",
                     right_extra_journal="UDEA_nombre revista o premio"
                     )

In [ ]:
newwos=drive_files.WOS_UDEA[drive_files.WOS_UDEA.Tipo!='UDEA']

In [ ]:
newwos.shape

In [ ]:
app_to_UDEA_PTJ_tot=newwos[newwos.Tipo.str.contains('UDEA')].reset_index(drop=True)
app_to_UDEA_PTJ_tot.shape

In [ ]:
new_UDEA_not_PTJ=newwos[~newwos.Tipo.str.contains('UDEA')].reset_index(drop=True)
new_UDEA_not_PTJ.shape[0]+app_to_UDEA_PTJ_tot.shape[0]

Update aunly and pass again

With the total UDEA_PTJ ~2500 runs againt WOS_SCI_SCP_PTJ.json  and add UDEA_authors info and extrapolates new ones ~500. We will have new ~3000 for a total of 11000~

In [ ]:
#qq=pd.read_json('WOS_SCI_SCP_PTJ.json.gz',compression='gzip')

In [ ]:
[ c for c in drive_files.biblio['WOS'].columns if c.find('UDEA_')>-1  ]

In [ ]:
new_UDEA_PTJ=app_to_UDEA_PTJ.append(app_to_UDEA_PTJ_2).append(app_to_UDEA_PTJ_tot).reset_index(drop=True)

In [ ]:
new_UDEA_PTJ.shape[0]+new_UDEA_not_PTJ.shape[0]

In [ ]:
#new_UDEA_PTJ=pd.read_json('new_UDEA_PTJ.json').reset_index(drop=True)

In [ ]:
UDEA_PTJ.shape

In [ ]:
[ c for c in UDEA_PTJ.columns if c.find('UDEA_')>-1  ]

In [ ]:
[ c for c in new_UDEA_PTJ.columns if c.find('UDEA_')>-1  ]

In [ ]:
print 1

In [ ]:
drive_files.load_biblio?

# TMP

The entries left are the ones where there are normalization problemas
* Bad written names
* Wrong affiliation in WOS
* Author appears in AU but not in C1

In [ ]:
i=0
tmpua=UDEA[UDEA.UDEA_authors.str[i].apply(lambda d: True if type(d)==dict else False)]

In [ ]:
tmpkk=tmpua[ tmpua.UDEA_authors.str[i].apply(lambda d: d.get('WOS_author')).str[0].apply( pd.isna) ]

In [ ]:
tmp=tmpkk[ tmpkk.authors_WOS.apply( lambda l: l if l and type(l)==list else pd.np.nan ).apply( 
    lambda l: True if type(l)==list else False) ][['authors_WOS','UDEA_authors','UDEA_autores','TI','DI','AU','C1']].reset_index(drop=True)#.loc[4].values

In [ ]:
tmp.shape

In [ ]:
i=204
tmp[i:i+1]#.UDEA_authors#.loc[0]

In [ ]:
tmp.UDEA_authors.loc[i]

In [ ]:
tmp.authors_WOS.loc[i]

In [ ]:
tmp.UDEA_autores.loc[i]

In [ ]:
tmp.TI.loc[i]

In [ ]:
print(tmp.AU.loc[i])

In [ ]:
print(tmp.C1.loc[i])

In [ ]:
x=tmp['authors_WOS'].reset_index(drop=True).loc[i]
y=tmp['UDEA_authors'].reset_index(drop=True).loc[i]

In [ ]:
x

In [ ]:
y

In [ ]:
i=0
y#[i]['NOMBRE COMPLETO'].split()

In [ ]:
y_keys=['PRIMER APELLIDO','NOMBRES','INICIALES','SEGUNDO APELLIDO','full_name']

In [ ]:
i=0
wos_names_list(y[i] ,y_keys)

In [ ]:
x

In [ ]:
[1,
  2]

In [ ]:
def combinewos(x,y,x_keys=['WOS_author','affiliation'],
                   y_keys=['PRIMER APELLIDO','NOMBRES','INICIALES','SEGUNDO APELLIDO','full_name'],
                   xy_keys=['WOS_author','WOS_affiliation']):
    if type(x)==list and type(y)==list:
        for dx in x:
            print( dx[ x_keys[0] ] )
            wos_name=unidecode.unidecode( dx[ x_keys[0] ] )
            WOS_affiliation= dx[x_keys[1]]
            print('Name True')
            for i in range( len(y) ):
                print('+'*20)
                print( wos_names_list(y[i] ,y_keys))
                print('+'*20)
                print(i,wos_name in wos_names_list(y[i] ,y_keys) )
                print('='*20)
                if wos_name.title() in wos_names_list(y[i] ,y_keys):
                    print('*'*20)
                    print(wos_name in wos_names_list(y[i] ,y_keys),':',wos_name)
                    print('*'*20)
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break
            # Try by buildong spanish-like names list                    
            #Try again but comparing full lists
            wos_name_to_list=wos_name.replace(',','').replace('-',' ').title().split()
            print("-"*20)
            for i in range( len(y) ):
                print('--',i)
                yi_to_list=unidecode.unidecode( y[i][y_keys[4]].title() )
                if yi_to_list:
                    yi_to_list=yi_to_list.split()
                else:
                    yi_to_list=[]
                print(i,wos_name_to_list,yi_to_list,  )    
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break
            #Try again but comparing full lists with initials
            wos_name_to_list=wos_name.replace(',','').replace('-',' ').title().split()
            for i in range( len(y) ):
                yi_to_list=[y[i][y_keys[0]],y[i][y_keys[3]] ]+y[i][y_keys[2]].split()
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break
            #Try again but comparing full lists with first first name and initial
            print(":"*20)
            for i in range( len(y) ):
                yi_to_list=[y[i][y_keys[0]],y[i][y_keys[3]],y[i][y_keys[1]].split()[0],
                              y[i][y_keys[2]].split()[-1]]
                print("::::",yi_to_list)
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break                    
            #Try again but comparing full lists with second first name and initial
            print(":"*20)
            for i in range( len(y) ):
                yi_to_list=[y[i][y_keys[0]],y[i][y_keys[3]],y[i][y_keys[1]].split()[-1],
                              y[i][y_keys[2]].split()[0]]
                print("::::",yi_to_list)
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break                    
                    
                    
                    
    return y

In [ ]:
combinewos(x,y)

In [ ]:
pd.np.setdiff1d(['Bastidas', 'Myriam'] ,['Bastidas', 'Acevedo', 'Miryam', 'Del', 'Socorro']).shape[0]

In [ ]:
800*3*3

In [ ]:
aunly

## TODO: Try to merge new WOS articles from old `'UDEA_authors'`

TODO: whithout split or merge

Split DataFrame into  "new" with UDEA authors not yet identified and "old" with full UDEA author info 

In [ ]:
UDEAnew=UDEA[UDEA.UDEA_authors.isna()].reset_index(drop=True)
UDEAold=UDEA[~UDEA.UDEA_authors.isna()].reset_index(drop=True)

### Merge old with news

Split "new" into the "Y" with WOS author info

In [ ]:
UDEAnewY=UDEAnew[ UDEAnew.authors_WOS.apply(lambda x: type(x)==list and len(x)>0) ].reset_index(drop=True)
UDEAnewN=UDEAnew[~UDEAnew.authors_WOS.apply(lambda x: type(x)==list and len(x)>0) ].reset_index(drop=True)

In [ ]:
(UDEAnewY.shape,'+',UDEAnewN.shape,'+',UDEAold.shape,'=',
   UDEAnewY.shape[0]+UDEAnewN.shape[0]+UDEAold.shape[0],'=',UDEA.shape[0])

In [ ]:
CHECK=False
if CHECK:
    pattern='Restrepo, D.'
    mask=extract_internal_value_of_a_dictionary_key_in_a_list_of_dictionaries(
                UDEAnewY,list_of_dictionaries='authors_WOS',dictionary_key='WOS_author').apply( 
                lambda x: pattern in x)
    UDEAnewY[mask].authors_WOS.reset_index(drop=True).values

In [ ]:
if CHECK:
    mask=mask_on_internal_value_of_a_dictionary_key_in_a_list_of_dictionaries(UDEAnewY,
            pattern='Restrepo, D.',list_of_dictionaries='authors_WOS',dictionary_key='WOS_author')                                                                       
    UDEAnewY[mask]['authors_WOS'].reset_index(drop=True).values

In [ ]:
#Extract a specific author with specific affiliation from the WOS author full info into a temporal column
i=0
aff_i=0
UDEAnewY['news_i_j_str']=UDEAnewY.authors_WOS.apply(lambda x: [ {'WOS_author':y.get('WOS_author'),
                                                   'affiliation':y.get('affiliation')[aff_i]} for y in x]
                                                   ).str[i].astype(str)
UDEAnewY['news_i_j_str'].loc[0]

Prepare trained data set with the j-th author: first UDEA author info into a temporal column

In [ ]:
j=0
auth_j=0
aff_j=0
UDEAold['UDEA_authors_j']=UDEAold['UDEA_authors'].str[j]

In [ ]:
#Extract a specific author with specific affiliation from the WOS author full info into a temporal column
UDEAold['news_i_j']=UDEAold['UDEA_authors'].apply(lambda l: [ {'WOS_author':d.get('WOS_author')[auth_j], 
                                           'affiliation':d.get('WOS_affiliation')[aff_j] } if d.get('WOS_author') 
                                         else pd.np.nan for d in l] ).str[j]#.astype(str)


In [ ]:
ma=UDEAold.dropna(subset=['news_i_j']).reset_index(drop=True)

### Quality critera search:
* Affiliation split(', ') > 3
* if Author find('.') → 'Author split(' ')  > 2 

In [ ]:
ma['news_i_j'].loc[2]

In [ ]:
#*** May be filter Homonymous
Homonymous=False
if Homonymous:
    ma=ma[~np.logical_and( ma['news_i_j'].apply(lambda d: d.get('affiliation') 
                                              if not pd.isna(d) else '' ).str.split(', ').apply(len)<=3,
                            ma['news_i_j'].apply(lambda d: d.get('WOS_author') 
                                              if not pd.isna(d) else '' ).str.split(' ').apply(len)<=2)]
#****

In [ ]:
ma['news_i_j_str']=ma['news_i_j'].astype(str)

In [ ]:
#Extract all the mathches for the author
kk=UDEAnewY.merge( ma[['news_i_j_str','news_i_j']],on='news_i_j_str',how='left')#.dropna().reset_index(drop=True)#.loc[5].values

In [ ]:
ma.UDEA_authors.str[i].loc[0]

In [ ]:
kk.shape,UDEAnewY.shape

In [ ]:
kkk=kk.dropna(subset=['news_i_j'])#.shape
kkk.shape